In [ ]:
import joblib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.compose import (
    ColumnTransformer,
    make_column_selector,
    make_column_transformer,
)
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    OneHotEncoder,
    OrdinalEncoder,
    RobustScaler,
    StandardScaler,
    MinMaxScaler,
)

# import sklearn kmeans
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import set_config
from src.helpers import helper_functions, load_data, visuals

In [ ]:
set_config(transform_output="pandas")

In [ ]:
df_housing_raw = load_data.load_housing_raw_data()

In [ ]:
project_path = helper_functions.get_project_path()

# Plan
1. splitting train-test
2. exploring data
3. data preparation pipeline (cleaning, imputing, feature engineering)
4. hyperparameter tuning
5. overfitting/underfitting check
6. evaluation on testing data

# Quick EDA to know how to stratify and split the data into train/test

When splitting our data, we want to make sure the training set is representative of the cases we want to generalize to. Otherwise, we would train machine learning models that would not make accurate predictions.
That is why we need to make sure the distribution of key features correlated to our target are preserved in the test set. By doing so, we are evaluating our machine learning models against representative data and hence, we can trust the quality of our models' predictions.

Splitting the data in this manner is called _stratified sampling_. To do so, we need to do some basic exploratory of our data. This is what we will do now.

In [ ]:
df_housing_raw.isna().sum()

In [ ]:
# plot a correlation matrix using seaborn and coolwarm colormap
sns.heatmap(df_housing_raw.corr(), square=True, annot=True, cmap="coolwarm")
# add title
plt.title("Correlation Matrix")
# save the plot and make sure the axis labels are not cut off
plt.savefig(project_path / "images" / "correlation_matrix.png", bbox_inches="tight")


plt.show()

2 insights from the correlation plot:
- the median_house_value (target) is quite correlated to the median_income, so we will use it to split the data (in a stratified manner)
- total_bedrooms has 207 missing values and is very correlated to households, so we will use it to fill the missing values with a customer sklearn transformer

In [ ]:
# plot the distribution of all the numerical features, add a title to the plot and save the plot to the images folder
df_housing_raw.hist(bins=50, figsize=(20, 10))
plt.suptitle("Distribution of numerical features", fontsize=16)
plt.savefig(project_path / "images" / "numerical_features_distribution.png", dpi=300)
plt.show()

In [ ]:
df_housing = df_housing_raw.copy()

In [ ]:
# bin median_income into 5 bins ([0, 1.5, 3, 4.5, 6, np.inf]) and plot the value counts
df_housing["median_income_bin"] = pd.cut(
    df_housing["median_income"],
    bins=[0, 1.5, 3, 4.5, 6, np.inf],
    labels=[1, 2, 3, 4, 5],
)
df_housing["median_income_bin"].value_counts().sort_index().plot(kind="bar")
plt.xticks(rotation=0)

# show each value on top of each bar (centered)
for index, value in enumerate(
    df_housing["median_income_bin"].value_counts().sort_index()
):
    plt.text(index, value, str(value), ha="center")

plt.title("Median income category")
plt.xlabel("Median income category")
plt.ylabel("Count")
plt.show()

# Split into train/test data

In [ ]:
df_housing["median_income_bin"] = pd.cut(
    df_housing["median_income"],
    bins=[0, 1.5, 3, 4.5, 6, np.inf],
    labels=[1, 2, 3, 4, 5],
)

In [ ]:
df_train, df_test = train_test_split(
    df_housing,
    test_size=0.2,
    random_state=42,
    stratify=df_housing["median_income_bin"],
)

df_train.drop("median_income_bin", axis=1, inplace=True)
df_test.drop("median_income_bin", axis=1, inplace=True)

# split into X and y
X_train = df_train.drop("median_house_value", axis=1)
y_train = df_train["median_house_value"].copy()

X_test = df_test.drop("median_house_value", axis=1)
y_test = df_test["median_house_value"].copy()

# More EDA (training set only)

In [ ]:
fig, ax = plt.subplots()
df_train.plot(
    kind="scatter",
    x="longitude",
    y="latitude",
    s=df_train["population"] / 50,
    c="median_house_value",
    cmap="jet",
    ax=ax,
    alpha=0.5,
    title="median_house_value geospatial distribution",
)
plt.savefig(project_path / "images/median_house_value_geospatial.png")

In [ ]:
fig, ax = plt.subplots()
df_train.plot(
    kind="scatter",
    x="median_income",
    y="median_house_value",
    alpha=0.5,
    title="median_house_value in relation to median_income",
    ax=ax,
)
plt.savefig(project_path / "images/house_value_vs_income.png")

In [ ]:
# plot the value counts of ocean_proximity
df_train["ocean_proximity"].value_counts().plot(kind="bar")
plt.xticks(rotation=0)
# add the value on top of each bar (centered) and add a title
for index, value in enumerate(df_train["ocean_proximity"].value_counts()):
    plt.text(index, value, str(value), ha="center")
plt.title("Ocean proximity")
plt.xlabel("Ocean proximity")
plt.ylabel("Count")
# save the plot to the images folder
plt.savefig(project_path / "images/ocean_proximity.png")

In [ ]:
# plot distribution of households with annotation of values for each bin
# make sure the axis labels are not cut off
fig, ax = plt.subplots()
pd.cut(
    df_housing["households"],
    bins=[
        0,
        100,
        200,
        300,
        400,
        500,
        600,
        700,
        800,
        900,
        1000,
        1100,
        1200,
        1300,
        np.inf,
    ],
).value_counts().sort_index().plot(kind="bar", ax=ax)
ax.set_title("Distribution of bins of 'households' feature")
ax.set_ylabel("Count")
for index, value in enumerate(
    pd.cut(
        df_housing["households"],
        bins=[
            0,
            100,
            200,
            300,
            400,
            500,
            600,
            700,
            800,
            900,
            1000,
            1100,
            1200,
            1300,
            np.inf,
        ],
    )
    .value_counts()
    .sort_index()
):
    plt.text(index, value, str(value), ha="center")
plt.savefig(project_path / "images/households_distribution.png" , bbox_inches="tight")

In [ ]:
# create a new column with adequate bin households
df_housing["households_bin"] = pd.cut(
    df_housing["households"],
    bins=[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, np.inf],
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
)

In [ ]:
# plot total_bedrooms grouped by households_bin
df_housing.groupby("households_bin")["total_bedrooms"].mean().plot(kind="bar")
# add the value (rounded nearest integer) on top of each bar (centered) and add a title
for index, value in enumerate(
    df_housing.groupby("households_bin")["total_bedrooms"].mean()
):
    plt.text(index, value, str(round(value)), ha="center")


plt.xticks(rotation=0)
plt.title("Average total_bedrooms per households_bin")
plt.xlabel("households_bin")
plt.ylabel("Average total_bedrooms")
# save the plot to the images folder
plt.savefig(project_path / "images/total_bedrooms_per_households_bin.png")
plt.show()

# Feature engineering

Let's create a customer imputer for total_bedrooms

In [ ]:
class GroupedImputer(BaseEstimator, TransformerMixin):
    """Custom imputer that fill missing values of a column with its median or mean by groups of by"""

    def __init__(self, variable, by):
        self.variable = variable
        self.by = by

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X[self.variable] = X.groupby(self.by)[self.variable].transform(
            lambda x: x.fillna(x.median())
        )
        return X

In [ ]:
# imputer_test = GroupedImputer(variable="total_bedrooms", by="households_bin")
# df_housing = imputer_test.fit_transform(df_housing)

Let's do some feature engineering with the location (lat and long)

In [ ]:
# custom transformer in scikit-learn that fits a Kmeans using the lat and long features and adds the distances to the clusters as new features
class KMeansTransformer(BaseEstimator, TransformerMixin):
    """Custom transformer that fits a Kmeans using the lat and long features and adds the distances to the clusters as new features"""

    def __init__(self, n_clusters=5, weights=None, pass_through=False):
        self.n_clusters = n_clusters
        self.weights = weights
        self.pass_through = pass_through

    def fit(self, X, y=None):
        if self.pass_through:
            return self
        self.kmeans = KMeans(n_clusters=self.n_clusters)
        self.kmeans.fit(X[["latitude", "longitude"]], sample_weight=self.weights)
        return self

    def transform(self, X, y=None):
        if self.pass_through:
            return X

        X["kmeans_cluster"] = self.kmeans.predict(X[["latitude", "longitude"]])
        for i in range(self.n_clusters):
            X[f"kmeans_distance_{i}"] = self.kmeans.transform(
                X[["latitude", "longitude"]]
            ).iloc[:, i]
        return X

Let's see how the clusters look

In [ ]:
# fit and transform the kmeans transformer on X_train
kmeans_transformer = KMeansTransformer(n_clusters=10, weights=y_train)
X_train = kmeans_transformer.fit_transform(X_train)

# plot the kmeans clusters and draw lines to the centroids for each cluster for each row.
fig, ax = plt.subplots(figsize=(20, 10))
X_train.plot(
    kind="scatter",
    x="longitude",
    y="latitude",
    c="kmeans_cluster",
    cmap="jet",
    alpha=0.5,
    ax=ax,
)
for i in range(kmeans_transformer.n_clusters):
    plt.plot(
        kmeans_transformer.kmeans.cluster_centers_[i, 1],
        kmeans_transformer.kmeans.cluster_centers_[i, 0],
        "kx",
    )
    plt.text(
        kmeans_transformer.kmeans.cluster_centers_[i, 1] + 0.01,
        kmeans_transformer.kmeans.cluster_centers_[i, 0] + 0.01,
        f"Cluster {i}",
        fontsize=12,
    )
# add a title
plt.title("Kmeans clusters")
# save the plot to the images folder
plt.savefig(project_path / "images/kmeans_clusters.png")

plt.show()

In [ ]:
# revert X_train to before KmeansTransformer was applied
X_train = X_train.drop(
    columns=[f"kmeans_distance_{i}" for i in range(kmeans_transformer.n_clusters)]
    + ["kmeans_cluster"]
)

In [ ]:
# create pipeline for the numerical and categorical features and add the KmeansTransformer to the numerical pipeline
num_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("kmeans", KMeansTransformer(n_clusters=10, weights=y_train)),
        ("scaler", StandardScaler()),
    ]
)
cat_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)


# create a column transformer that applies the numerical and categorical pipeline to the numerical and categorical features
preprocessor = ColumnTransformer(
    [
        ("num", num_pipeline, make_column_selector(dtype_include=np.number)),
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ]
)


# create a pipeline that applies the preprocessor and a random forest regressor
pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("regressor", RandomForestRegressor()),
    ]
)

In [ ]:
# show the pipeline
pipeline

In [ ]:
# slice the pipeline to only the num preprocessor
num_pipeline = pipeline.steps[0][1].transformers[0][1]
# remove (in place) the kmeans transformer from the num pipeline
# num_pipeline.steps.pop(1)

# column transformer that only applies the num pipeline to the numerical features
num_pipeline = ColumnTransformer(
    [
        ("num", num_pipeline, make_column_selector(dtype_include=np.number)),  
    ]
)



# fit X_train using num_pipeline
num_pipeline.fit_transform(X_train, y_train)

# num_pipeline


In [ ]:
# imputer = SimpleImputer(strategy="median")
# kmeans = KMeansTransformer(n_clusters=10, weights=y_train)
# X_train["total_bedrooms"] = imputer.fit_transform(X_train[["total_bedrooms"]], y_train)
# display(X_train)
# X_train = kmeans.fit_transform(X_train, y_train)
# X_train

In [ ]:
# insert a custom debugger transformer in the pipeline to inspect the data at each step
class Debugger(BaseEstimator, TransformerMixin):
    """Custom transformer that prints the shape of the data at each step"""

    def __init__(self, name=""):
        self.name = name

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        print(f"{self.name} transform: {X.shape}")
        return X

# insert the debugger transformer in the pipeline just after num SimpleImputer
pipeline.steps[0][1].transformers[0][1].steps.insert(
    1, ("debugger", Debugger(name="num imputer"))
)
# insert the debugger transformer in the pipeline just after num KmeansTransformer
pipeline.steps[0][1].transformers[0][1].steps.insert(
    3, ("debugger2", Debugger(name="num kmeans"))
)
# insert the debugger transformer in the pipeline just after num StandardScaler
pipeline.steps[0][1].transformers[0][1].steps.insert(
    5, ("debugger3", Debugger(name="num scaler"))
)



In [ ]:
pipeline

In [ ]:
test = Pipeline(pipeline.steps[:-1])

In [ ]:
test

In [ ]:
test.fit(X_train, y_train)

In [ ]:
# fit the pipeline on X_train and y_train
pipeline.fit(X_train, y_train)

# show the score of the pipeline on X_train and y_train and X_test and y_test
print(f"Train score: {pipeline.score(X_train, y_train):.3f}")
print(f"Test score: {pipeline.score(X_test, y_test):.3f}")


# create a dataframe with the feature importances
df_feature_importances = pd.DataFrame(
    {
        "feature": pipeline.named_steps["preprocessor"]
        .transformers_[0][1]["kmeans"]
        .kmeans.cluster_centers_.flatten(),
        "importance": pipeline.named_steps["regressor"].feature_importances_,
    }
)

# plot the feature importances
fig, ax = plt.subplots(figsize=(20, 10))
df_feature_importances.plot(kind="bar", x="feature", y="importance", ax=ax)
# add a title
plt.title("Feature importances")
# save the plot to the images folder
plt.savefig(project_path / "images/feature_importances.png")

plt.show()

# create a dataframe with the predictions on X_train
df_predictions = pd.DataFrame(
    {
        "prediction": pipeline.predict(X_train),
        "actual": y_train,
    }
)

# plot the predictions on X_train
fig, ax = plt.subplots(figsize=(20, 10))
df_predictions.plot(kind="scatter", x="prediction", y="actual", ax=ax)
# add a title
plt.title("Predictions on X_train")
# save the plot to the images folder
plt.savefig(project_path / "images/predictions_on_X_train.png")

plt.show()

# create a dataframe with the residuals on X_train
df_residuals = pd.DataFrame(
    {
        "residual": pipeline.predict(X_train) - y_train,
        "actual": y_train,
    }
)

# plot the residuals on X_train
fig, ax = plt.subplots(figsize=(20, 10))
df_residuals.plot(kind="scatter", x="residual", y="actual", ax=ax)
# add a title
plt.title("Residuals on X_train")
# save the plot to the images folder
plt.savefig(project_path / "images/residuals_on_X_train.png")

plt.show()

In [ ]:
# parameters grid to search for the best hyperparameters for the random forest regressor.
# switch on and off the KmeansTransformer by setting pass_through to True or False

param_grid_preprocessor = {
    # imputer: either SimpleImputer or GroupedImputer or KNNImputer
    "preprocessor__num__imputer": [
        SimpleImputer(strategy="median"),
        GroupedImputer(variable="total_bedrooms", by="households_bin"),
        KNNImputer(),
    ],
    # scaler: either StandardScaler or MinMaxScaler or RobustScaler
    "preprocessor__num__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],
}

param_grid_kmeans = {
    "preprocessor__num__kmeans__n_clusters": [5, 10],
    "preprocessor__num__kmeans__weights": [None, y_train],
}

param_grid_rf = {
    "regressor__n_estimators": [100, 200, 300],
    "regressor__max_depth": [None, 5, 10],
    "regressor__min_samples_split": [2, 5, 10],
    "regressor__min_samples_leaf": [1, 2, 4],
}


param_grid_full = [
    {
        # do not use the KmeansTransformer
        "preprocessor__num__kmeans__pass_through": True,
        **param_grid_preprocessor,
        **param_grid_rf,
    },
    # use the KmeansTransformer
    {
        "preprocessor__num__kmeans__pass_through": False,
        **param_grid_preprocessor,
        **param_grid_kmeans,
        **param_grid_rf,
    },
]


# create a grid search with 5-fold cross validation
grid_search = GridSearchCV(
    pipeline,
    param_grid_full,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    verbose=2,
)

In [ ]:
%%time

# fit the grid search on the training data
grid_search.fit(X_train, y_train)



In [ ]:
# show the results in a dataframe (drop useless columns)
results = pd.DataFrame(grid_search.cv_results_).drop(
    columns=[
        "mean_fit_time",
        "std_fit_time",
        "mean_score_time",
        "std_score_time",
        "params",
        "split0_test_score",
        "split1_test_score",
        "split2_test_score",
        "split3_test_score",
        "split4_test_score",
        "rank_test_score",
    ]
)

# show the results
results

In [ ]:
# function to save the results to a excel file in models\tuning_results
def save_results(results, filename):
    results.to_excel(
        project_path / f"models/tuning_results/{filename}.xlsx", index=False
    )


# save the results
save_results(results, "second_tuning")

In [ ]:
grid_search.best_params_

In [ ]:
results = pd.DataFrame(grid_search.cv_results_).sort_values("rank_test_score")
results = results[
    [
        "param_model",
        "param_preprocessing__cat__onehotencoder",
        "param_preprocessing__num__simpleimputer",
        "param_preprocessing__num__standardscaler",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
]
results

In [ ]:
final_model = grid_search.best_estimator_

In [ ]:
X_test = df_test.drop("median_house_value", axis=1)
y_test = df_test["median_house_value"]

In [ ]:
final_predictions = final_model.predict(X_test)
final_rmse = mean_squared_error(y_test, final_predictions, squared=False)
final_rmse

In [ ]:
results.to_excel(
    project_path / "models" / "tuning_results" / "first_tuning" / "first_tuning.xlsx",
    index=False,
)

In [ ]:
best_model = joblib.dump(
    grid_search.best_estimator_,
)